#How to develop deep learning models with keras
- [Your First Deep Learning Project in Python with Keras Step-By-Step](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/), by [Jason Brownlee](https://machinelearningmastery.com/author/jasonb/) on July 24, 2019 in [Deep Learning](https://machinelearningmastery.com/category/deep-learning/), last updated Oct 13, 2021
- [TensorFlow 2 Tutorial: Get Started in Deep Learning With tf.keras](https://machinelearningmastery.com/tensorflow-tutorial-deep-learning-with-tf-keras/) by [Jason Brownlee](https://machinelearningmastery.com/author/jasonb/) on December 19, 2019 in [Deep Learning](https://machinelearningmastery.com/category/deep-learning/)
- Excerpt by Ivan HP Lin (220428-done)



# A. Keras Tutorial Overview

The steps you are going to cover in this tutorial are as follows:

1. Load Data.
2. Define Keras Model.
3. Compile Keras Model.
4. Fit Keras Model.
5. Evaluate Keras Model.
6. Tie It All Together.
7. Make Predictions

## **A.1 Load Data**

We will use the NumPy library to load our dataset and we will use two classes from the Keras library to define our model.

In [1]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
...

Ellipsis

In this Keras tutorial, we are going to use the Pima Indians onset of diabetes dataset. This is a standard machine learning dataset from the UCI Machine Learning repository. It describes patient medical record data for Pima Indians and whether they had an onset of diabetes within five years.

As such, it is a binary classification problem (onset of diabetes as 1 or not as 0). All of the input variables that describe each patient are numerical. This makes it easy to use directly with neural networks that expect numerical input and output values, and ideal for our first neural network in Keras.

The dataset is available from here:

- [Dataset CSV File (pima-indians-diabetes.csv)](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv)
- [Dataset Details](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names)

In [2]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv

--2022-04-27 06:49:01--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘pima-indians-diabetes.data.csv’

pima-indians-diabet 100%[===================>]  22.73K  --.-KB/s    in 0.002s  

2022-04-27 06:49:01 (11.1 MB/s) - ‘pima-indians-diabetes.data.csv’ saved [23278/23278]



In [3]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names

--2022-04-27 06:49:01--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3066 (3.0K) [text/plain]
Saving to: ‘pima-indians-diabetes.names’

pima-indians-diabet 100%[===================>]   2.99K  --.-KB/s    in 0s      

2022-04-27 06:49:01 (32.1 MB/s) - ‘pima-indians-diabetes.names’ saved [3066/3066]



We can now load the file as a matrix of numbers using the NumPy function [loadtxt()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html).

There are eight input variables and one output variable (the last column). We will be learning a model to map rows of input variables (X) to an output variable (y), which we often summarize as $\bf y = f(X)$.

The variables can be summarized as follows:

* Input Variables ($\bf X$):
  1. Number of times pregnant
  2.Plasma glucose concentration a 2 hours in an oral glucose tolerance test
  3. Diastolic blood pressure (mm Hg)
  4.Triceps skin fold thickness (mm)
  5. 2-Hour serum insulin (mu U/ml)
  6. Body mass index (weight in kg/(height in m)^2)
  7. Diabetes pedigree function
  8. Age (years)

* Output Variables ($\bf y$):
  1. Class variable (0 or 1)

Once the CSV file is loaded into memory, we can split the columns of data into input and output variables.

The data will be stored in a 2D array where the first dimension is rows and the second dimension is columns, e.g. [rows, columns].

We can split the array into two arrays by selecting subsets of columns using the standard NumPy [**slice operator**](https://machinelearningmastery.com/index-slice-reshape-numpy-arrays-machine-learning-python/) or “**:**” We can select the first 8 columns from index 0 to index 7 via the slice 0:8. We can then select the output column (the 9th variable) via index 8.

* Reference: [**How to Index, Slice and Reshape NumPy Arrays for Machine Learning in Python**](https://machinelearningmastery.com/index-slice-reshape-numpy-arrays-machine-learning-python/)

In [4]:
...
# load the dataset
dataset = loadtxt('pima-indians-diabetes.data.csv', delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]
...

Ellipsis

## **A.2 Define Keras Model**

Models in Keras are defined as a sequence of layers.

We create a [***Sequential model***](https://keras.io/models/sequential/) and add layers one at a time until we are happy with our network architecture.

The first thing to get right is to ensure the input layer has the right number of input features. This can be specified when creating the first layer with the input_dim argument and setting it to 8 for the 8 input variables.

In this example, we will use a fully-connected network structure with three layers.

Fully connected layers are defined using the [**Dense class**](https://keras.io/layers/core/). We can specify the number of neurons or nodes in the layer as the first argument, and specify the activation function using the **activation** argument.

We will use the [rectified linear unit activation function](https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/) referred to as ReLU on the first two layers and the Sigmoid function in the output layer.

It used to be the case that Sigmoid and Tanh activation functions were preferred for all layers. These days, better performance is achieved using the ReLU activation function. We use a sigmoid on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5.

We can piece it all together by adding each layer:

* The model expects rows of data with 8 variables (the input_dim=8 argument)
* The first hidden layer has 12 nodes and uses the relu activation function.
* The second hidden layer has 8 nodes and uses the relu activation function.
* The output layer has one node and uses the sigmoid activation function.

In [5]:
...
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
...

Ellipsis

## **A.3 Compile Keras Model**

Compiling the model uses the efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware, such as CPU or GPU or even distributed.

When compiling, we must specify some additional properties required when training the network. Remember training a network means finding the best set of weights to map inputs to outputs in our dataset.

1. We must specify the **loss function** to use to evaluate a set of weights, the optimizer is used to search through different weights for the network and any optional metrics we would like to collect and report during training.

2. In this case, we will use **cross entropy** as the **loss** argument. This loss is for a binary classification problems and is defined in Keras as “**binary_crossentropy**“. You can learn more about choosing loss functions based on your problem here:
  * [How to Choose Loss Functions When Training Deep Learning Neural Networks](https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/)

4. We will define the **optimizer** as the efficient stochastic gradient descent algorithm “**adam**“. It automatically tunes itself and gives good results in a wide range of problems. To learn more about the Adam version of stochastic gradient descent see the post:
  * [Gentle Introduction to the Adam Optimization Algorithm for Deep Learning](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/)

5. Finally, because it is a classification problem, we will collect and report the classification accuracy, defined via the **metrics** argument.

In [6]:
...
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
...

Ellipsis

No GPU is required for this example, but if you’re interested in how to run large models on GPU hardware cheaply in the cloud, see this post:

- [How to Setup Amazon AWS EC2 GPUs to Train Keras Deep Learning Models](https://machinelearningmastery.com/develop-evaluate-large-deep-learning-models-keras-amazon-web-services/)

## A.4 **Fit Keras Model**

We can train or fit our model on our loaded data by calling the **fit()** function on the model.

Training occurs over epochs and each epoch is split into batches.

* **Epoch**: One pass through all of the rows in the training dataset.
* **Batch**: One or more samples considered by the model within an epoch before weights are updated.

One epoch is comprised of one or more batches, based on the chosen batch size and the model is fit for many epochs. For more on the difference between epochs and batches, see the post:

- [What is the Difference Between a Batch and an Epoch in a Neural Network?](https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/)

The training process will run for a fixed number of iterations through the dataset called **epochs**, that we must specify using the epochs argument. We must also set the number of dataset rows that are considered before the model weights are updated within each epoch, called the **batch size** and set using the **batch_size** argument.

In [14]:
# For this problem, we will run for a small number of epochs (150) and use a relatively small batch size of 10.
...
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10, verbose='auto')
...

Epoch 1/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4988 - accuracy: 0.7513
Epoch 2/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4962 - accuracy: 0.7643
Epoch 3/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4937 - accuracy: 0.7578
Epoch 4/150
77/77 [==============================] - 0s 2ms/step - loss: 0.5077 - accuracy: 0.7487
Epoch 5/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4935 - accuracy: 0.7513
Epoch 6/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4921 - accuracy: 0.7604
Epoch 7/150
77/77 [==============================] - 0s 1ms/step - loss: 0.5018 - accuracy: 0.7513
Epoch 8/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4928 - accuracy: 0.7526
Epoch 9/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4941 - accuracy: 0.7565
Epoch 10/150
77/77 [==============================] - 0s 2ms/step - loss: 0.4892 - accuracy: 0.7682
Epoch 11/

## A.5 **Evaluate Keras Model**

We have trained our neural network on the entire dataset and we can evaluate the performance of the network on the same dataset.

This will only give us an idea of how well we have modeled the dataset (e.g. train accuracy), but no idea of how well the algorithm might perform on new data. We have done this for simplicity, but ideally, you could separate your data into train and test datasets for training and evaluation of your model.

You can evaluate your model on your training dataset using the **evaluate()** function on your model and pass it the same input and output used to train the model.

This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

The **evaluate()** function will return a list with two values. 
1. The first will be the loss of the model on the dataset, and 
2. The second will be the accuracy of the model on the dataset. 

We are only interested in reporting the accuracy, so we will ignore the loss value.

In [15]:
...
# evaluate the keras model
_, accuracy = model.evaluate(X, y, verbose=2)
print('Accuracy: %.2f' % (accuracy*100))

24/24 - 0s - loss: 0.4732 - accuracy: 0.7760 - 38ms/epoch - 2ms/step
Accuracy: 77.60


## A.6 **Tie It All Together**

In [18]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
# load the dataset
dataset = loadtxt('pima-indians-diabetes.data.csv', delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10, verbose='auto')
# evaluate the keras model
_, accuracy = model.evaluate(X, y, verbose='auto')
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/150
77/77 [==============================] - 1s 2ms/step - loss: 6.0138 - accuracy: 0.3672
Epoch 2/150
77/77 [==============================] - 0s 2ms/step - loss: 1.2277 - accuracy: 0.4349
Epoch 3/150
77/77 [==============================] - 0s 2ms/step - loss: 0.8061 - accuracy: 0.5391
Epoch 4/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7122 - accuracy: 0.5872
Epoch 5/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6783 - accuracy: 0.6042
Epoch 6/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6504 - accuracy: 0.6328
Epoch 7/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6361 - accuracy: 0.6576
Epoch 8/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6251 - accuracy: 0.6680
Epoch 9/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6249 - accuracy: 0.6602
Epoch 10/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6155 - accuracy: 0.6628
Epoch 11/

## A.7 **Make Predictions**

Making predictions is as easy as calling the **predict()** function on the model. We are using a sigmoid activation function on the output layer, so the predictions will be a probability in the range between 0 and 1. We can easily convert them into a crisp binary prediction for this classification task by rounding them.

For example,
```
...
# make probability predictions with the model
predictions = model.predict(X)
# round predictions 
rounded = [round(x[0]) for x in predictions]
```
Alternately, we can convert the probability into 0 or 1 to predict crisp classes directly, for example:
```
...
# make class predictions with the model
predictions = (model.predict(X) > 0.5).astype(int)
```

Completely code example as below.

If you would like to know more about how to make predictions with Keras models, see the post:

* [How to Make Predictions with Keras](https://machinelearningmastery.com/how-to-make-classification-and-regression-predictions-for-deep-learning-models-in-keras/)

In [20]:
# first neural network with keras make predictions
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
# load the dataset
dataset = loadtxt('pima-indians-diabetes.data.csv', delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10, verbose=0)
# make class predictions with the model
predictions = (model.predict(X) > 0.5).astype(int)
# summarize the first 10 cases
for i in range(10):
	print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 1 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 1 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 1 (expected 1)
[5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0] => 0 (expected 0)
[3.0, 78.0, 50.0, 32.0, 88.0, 31.0, 0.248, 26.0] => 0 (expected 1)
[10.0, 115.0, 0.0, 0.0, 0.0, 35.3, 0.134, 29.0] => 1 (expected 0)
[2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0] => 1 (expected 1)
[8.0, 125.0, 96.0, 0.0, 0.0, 0.0, 0.232, 54.0] => 0 (expected 1)


# B. **Keras Tutorial Extensions**

This section provides some extensions to this tutorial that you might want to explore.

* **Tune the Model**. Change the configuration of the model or training process and see if you can improve the performance of the model, e.g. achieve better than 76% accuracy.
* **[Save the Model](https://machinelearningmastery.com/save-load-keras-deep-learning-models/)**. Update the tutorial to save the model to file, then load it later and use it to make predictions (see this [tutorial](https://machinelearningmastery.com/save-load-keras-deep-learning-models/)).
* **[Summarize the Model](https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/)**. Update the tutorial to summarize the model and create a plot of model layers (see this [tutorial](https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/)).
* **Separate Train and Test Datasets**. Split the loaded dataset into a train and test set (split based on rows) and use one set to train the model and the other set to estimate the performance of the model on new data.
* **[Plot Learning Curves](https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/)**. The fit() function returns a history object that summarizes the loss and accuracy at the end of each epoch. Create line plots of this data, called [learning curves](https://machinelearningmastery.com/learning-curves-for-diagnosing-machine-learning-model-performance/) (see this [tutorial](https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/)).
* **Learn a New Dataset**. Update the tutorial to use a different tabular dataset, perhaps from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php).
* **[Use Functional API](https://machinelearningmastery.com/keras-functional-api-deep-learning/)**. Update the tutorial to use the Keras Functional API for defining the model (see this [tutorial](https://machinelearningmastery.com/keras-functional-api-deep-learning/)).

## Related Tutorials

* [5 Step Life-Cycle for Neural Network Models in Keras](https://machinelearningmastery.com/5-step-life-cycle-neural-network-models-keras/)
* [Multi-Class Classification Tutorial with the Keras Deep Learning Library](https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/)
* [Regression Tutorial with the Keras Deep Learning Library in Python](https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/)
* [How to Grid Search Hyperparameters for Deep Learning Models in Python With Keras](https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/)